## Network analysis with ssb-gis-utils

Network analysis with igraph, integrated with geopandas.

The package supports three types of network analysis:
- od_cost_matrix: fast many-to-many travel times/distances
- shortest_path: returns the geometry of the lowest-cost paths.
- service_area: returns the roads that can be reached within one or more impedances.

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
while "gis_utils" not in os.listdir():
    os.chdir("../")

pd.options.mode.chained_assignment = None # ignore SettingWithCopyWarning

import gis_utils as gu
# gu.__version__

In [ ]:
points = gpd.read_parquet(r"C:\Users\ort\OneDrive - Statistisk sentralbyrå\data\tilfeldige_adresser_1000.parquet")
p = points.iloc[[0]]

In [ ]:
roads = gpd.read_parquet(r"C:/Users/ort/OneDrive - Statistisk sentralbyrå/data/vegdata/veger_oslo_og_naboer_2022.parquet")
roads = roads[["oneway", "drivetime_fw", "drivetime_bw", "geometry"]]
roads.head(3)

## The Network

In [ ]:
nw = gu.Network(roads)
nw

In [ ]:
nw.gdf.head(3)

The network class includes methods for customising the road data. More about this further down in this notebook.

In [ ]:
nw = (nw
.close_network_holes(1.5)
.remove_isolated()
#.cut_lines(25)
)
nw

For directed network analysis, the DirectedNetwork class can be used. This inherits all methods from the Network class, and also includes methods for making a directed network.

In [ ]:
nw = gu.DirectedNetwork(roads).remove_isolated()
nw

The above warning suggests that the data might not be directed yet. This is correct. The roads going both ways, only appear once, and the roads going backwards, have to be turned around. 

This can be done in the make_directed_network method. 

In [ ]:
nw2 = nw.copy()
nw2 = nw2.make_directed_network(
        direction_col = "oneway",
        direction_vals_bft = ("B", "FT", "TF"),
        speed_col = None,
        minute_cols = ("drivetime_fw", "drivetime_bw"),
        flat_speed = None,
        )
nw2

The roads now have almost twice as many rows, since most roads are bidirectional in this network.

OpenStreetMap road data and Norwegian road network can be customised with custom methods, where the default parameters should give the correct results:

In [ ]:
# nw.make_directed_network_osm()

In [ ]:
nw = nw.make_directed_network_norway()
nw

## NetworkAnalysis

In [ ]:
nwa = gu.NetworkAnalysis(nw, cost="minutes")
nwa

In [ ]:
od = nwa.od_cost_matrix(points, points, id_col="idx")
od

In [ ]:
gu.qtm(
    nwa.od_cost_matrix(points.sample(1), points, lines=True),
    "minutes", title="Travel time (minutes) from 1 to 1000 addresses.", k=7
    )

In [ ]:
sp = nwa.shortest_path(points.iloc[[0]], points.sample(100), id_col="idx")

gu.qtm(sp)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

def chop_cmap_frac(cmap: LinearSegmentedColormap, frac: float) -> LinearSegmentedColormap:
    """Chops off the beginning `frac` fraction of a colormap."""
    cmap = plt.get_cmap(cmap)
    cmap_as_array = cmap(np.arange(256))
    cmap_as_array = cmap_as_array[int(frac * len(cmap_as_array)):]
    return LinearSegmentedColormap.from_list(cmap.name + f"_frac{frac}", cmap_as_array)

cmap = chop_cmap_frac("RdPu", 0.2)

sp = nwa.shortest_path(
    points.sample(150), points.sample(150), 
    summarise=True
)

gu.qtm(sp, "n", scheme="naturalbreaks", k=9, cmap=cmap, title="Number of times each road was used.")

In [ ]:
sa = nwa.service_area(points.sample(5), impedance=(5,10,15), id_col="idx")
sa

In [ ]:
sa = nwa.service_area(points.iloc[[0]], impedance=np.arange(1, 11))
sa = sa.sort_values("minutes", ascending=False)
gu.qtm(sa, "minutes", k=9)

Set dissolve=False to get each road segment returned, one for each service area that uses the segment. If you have a lot of overlapping service areas that are to be dissolved in the end, removing duplicates first makes things a whole lot faster.

In [ ]:
sa = nwa.service_area(points.sample(250), impedance=5, dissolve=False)

print(len(sa))

sa = sa.drop_duplicates(["source", "target"])

print(len(sa))

gu.qtm(sa)

### Customising the network

In [ ]:
nw = gu.DirectedNetwork(roads)
nw

In [ ]:
nw = nw.get_largest_component()

gu.clipmap(
    nw.gdf, points.iloc[[0]].buffer(1000),
    column="connected", 
    scheme="equalinterval", 
    cmap="bwr",
    explore=False,
    )

The isolated networks can be removed by setting remove=True. Or use the remove_isolated method:

In [ ]:
nw = nw.remove_isolated()

gu.clipmap(
    nw.gdf, points.iloc[[0]].buffer(1000),
    explore=False,
    )

nw

If your road data has small gaps between the segments, these can be populated with straight lines:

In [ ]:
nw = nw.close_network_holes(max_dist=1.5) # meters
nw

In [ ]:
nw = nw.cut_lines(25) # meters
nw